In [1]:
import os
import shutil
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')
folder_slope = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Slopes')
folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# open list of glaciers
path_data = os.path.join(folder_save, "Final_Data_Table.csv")
rgi_df = pd.read_csv(path_data)

In [3]:
rgi_df

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,hyps_aar_360,hyps_aar_380,hyps_aar_400,hyps_aar_420,hyps_aar_440,hyps_aar_460,hyps_aar_480,hyps_aar_500,hyps_aar_cmip,delta_ela_cmip
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120000,63.381000,1,2,3.681,1273,...,0.0174,0.0078,0.0026,0.0011,0.0004,0.0000,0.0000,0.0000,0.4362,101
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057000,63.551000,1,2,2.573,1494,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0495,112
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244000,63.571000,1,2,10.470,1201,...,0.2050,0.1974,0.1898,0.1820,0.1754,0.1687,0.1620,0.1536,0.3342,101
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170000,63.513000,1,2,3.806,1280,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,112
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651000,63.366000,1,2,2.802,1318,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0233,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,RGI60-01.27103,G225914E58943N,20050811,-9999999,-134.086000,58.943000,1,6,86.656,873,...,0.0086,0.0068,0.0054,0.0044,0.0035,0.0025,0.0015,0.0007,0.0995,214
2998,RGI60-01.27105,G227608E57164N,20040810,-9999999,-132.392000,57.164000,1,6,131.574,507,...,0.0618,0.0581,0.0543,0.0506,0.0462,0.0417,0.0351,0.0285,0.0909,249
2999,RGI60-01.27106,G220930E60344N,20100814,-9999999,-139.070000,60.344000,1,5,2.758,3481,...,0.1628,0.1481,0.1339,0.1169,0.1031,0.0875,0.0713,0.0569,0.8489,99
3000,RGI60-01.27108,G216273E60232N,20100910,-9999999,-143.726807,60.231728,1,5,534.228,3,...,0.0321,0.0310,0.0299,0.0287,0.0275,0.0265,0.0255,0.0245,0.0531,139


In [4]:
# iterate through each RGI we have data for
rgi_names = list(rgi_df['RGIId'].values)

for idx,rgiid in enumerate(rgi_names):
    # print(idx)
    
    ##### copy 5-year average rgi to folders
    
    # define source/sink paths
    source_path = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA_final_infill.tif")
    sink_path = os.path.join(folder_AGVA, 'Data Release', 'By Year', 'Average', f"{rgiid}_AA_2018-2022-average.tif")
    
    # check if file exists:
    if os.path.exists(source_path):
        
        # copy
        shutil.copy(source_path, sink_path)
    
    else:
        print(source_path, "does not exist")
    
    
    
    ##### copy individual years to folders
    
    for y in [2018,2019,2020,2021,2022]:
        
        # define source/sink paths
        source_path = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA_final_infill.tif")
        sink_path = os.path.join(folder_AGVA, 'Data Release', 'By Year', str(y), f"{rgiid}_AA_{y}.tif")
    
        # check if file exists:
        if os.path.exists(source_path):

            # copy
            shutil.copy(source_path, sink_path)
print("Done!")

Done!


In [5]:
### redo like above, but make a folder for each rgi

# iterate through each RGI we have data for
rgi_names = list(rgi_df['RGIId'].values)

for rgiid in rgi_names:
    
    # define folder name
    rgi_folder = os.path.join(folder_AGVA, 'Data Release', 'By RGI', rgiid)

    # check if this folder exists. if it does, delete and remake
    if os.path.exists(rgi_folder):
        os.rmdir(rgi_folder) 
    
    # remake empty folder
    os.makedirs(rgi_folder)

    

    
    ##### copy 5-year average product to folder
    
    # define source/sink paths
    source_path = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA_final_infill.tif")
    sink_path = os.path.join(rgi_folder, f"{rgiid}_AA_2018-2022-average.tif")
    
    # check if file exists:
    if os.path.exists(source_path):
        
        # copy
        shutil.copy(source_path, sink_path)
    
    else:
        print(source_path, "does not exist")
    
    
    
    ##### copy individual years to folders
    
    for y in [2018,2019,2020,2021,2022]:
        
        # define source/sink paths
        source_path = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA_final_infill.tif")
        sink_path = os.path.join(rgi_folder, f"{rgiid}_AA_{y}.tif")
    
        # check if file exists:
        if os.path.exists(source_path):

            # copy
            shutil.copy(source_path, sink_path)
print("Done!")

Done!


In [5]:
### last, make a csv holding info for each glacier
# let's maek a separate csv for each year
# and a csv for the average

# average: ela, aa, aar, off-glacier flag
# annual: ela, aar, % observed, off-glacier flag, date

# function that will recode values to be consistent
def recode(i):
    
    possible_nans = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01',
                      -9999, "-9999", np.nan, 'np.nan', 'nan']
    nan_to_use = "-9999"
    
    # true and false to 0/1
    if i=="True": return_value = "1"
    elif i=='False': return_value = "0"
    elif i in possible_nans: return_value = nan_to_use
    elif pd.isna(i): return_value = nan_to_use
    else: return_value = i
    
    return str(return_value)


### lists to hold data
d_avg = []
d_years = [[],[],[],[],[]]

# for each rgiid
rgi_names = list(rgi_df['RGIId'].values)
for rgiid in rgi_names:
    
    # grab the data for this glacier
    this_row = rgi_df[rgi_df['RGIId']==rgiid].fillna("-9999")
    
    # initialize dic to hold avg info
    rgi_dic = {}
    
    # get average data
    rgi_dic['RGIId_v6'] = this_row['RGIId'].values[0]
    rgi_dic['ela_2018-2022-avg'] = recode(this_row['ela_avg_product'].values[0])
    rgi_dic['aar_2018-2022-avg'] = recode(this_row['aar_infill_avg_product'].values[0])
    rgi_dic['off_glacier_2018-2022-avg'] = recode(this_row['off_glacier_avg_product'].values[0])
    d_avg.append(rgi_dic)
    
    # get each year's annual hata
    for idx,y in enumerate([2018,2019,2020,2021,2022]):
        
        # initialize dic to hold this year's info
        rgi_dic = {}
        
        # get data
        rgi_dic['RGIId_v6'] = this_row['RGIId'].values[0]
        rgi_dic[f'ela_{y}'] = recode(this_row[f'ela_{y}'].values[0])
        rgi_dic[f'aar_{y}'] = recode(this_row[f'aar_{y}'].values[0])
        rgi_dic[f'date_{y}'] = recode(this_row[f'date_{y}'].values[0])
        rgi_dic[f'off_glacier_{y}'] = recode(this_row[f'off_glacier_{y}'].values[0])
        
        # append to lists
        d_years[idx].append(rgi_dic)

# make to dfs
df_avg = pd.DataFrame(d_avg)
df_18 = pd.DataFrame(d_years[0])
df_19 = pd.DataFrame(d_years[1])
df_20 = pd.DataFrame(d_years[2])
df_21 = pd.DataFrame(d_years[3])
df_22 = pd.DataFrame(d_years[4])

print('Done!')

Done!


In [6]:
# save dfs
out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2018-2022-average.csv")
df_avg.to_csv(out_path, index=False)

out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2018.csv")
df_18.to_csv(out_path, index=False)

out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2019.csv")
df_19.to_csv(out_path, index=False)

out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2020.csv")
df_20.to_csv(out_path, index=False)

out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2021.csv")
df_21.to_csv(out_path, index=False)

out_path = os.path.join(folder_AGVA, 'Data Release', 'csv', f"2022.csv")
df_22.to_csv(out_path, index=False)

In [9]:
this_row

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,hyps_aar_360,hyps_aar_380,hyps_aar_400,hyps_aar_420,hyps_aar_440,hyps_aar_460,hyps_aar_480,hyps_aar_500,hyps_aar_cmip,delta_ela_cmip
3001,RGI60-01.27110,G230488E54596N,20049999,20069999,-129.512,54.5959,1,6,3.564,980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196
